In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from IPython.display import display, HTML

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
properties = pd.read_csv("../input/bb020fc4-9-dataset/Dataset/Property dataset/Properties.csv")

In [ ]:
propertiesz = pd.read_csv("../input/bb020fc4-9-dataset/Dataset/Property dataset/Properties.csv")

In [ ]:
test = pd.read_csv("../input/bb020fc4-9-dataset/Dataset/Test dataset/Test_Data.csv")

In [ ]:
train_acc = pd.read_csv("../input/bb020fc4-9-dataset/Dataset/training dataset/Accounts.csv")
train_opp = pd.read_csv("../input/bb020fc4-9-dataset/Dataset/training dataset/Opportunities.csv")
train_deal= pd.read_csv("../input/bb020fc4-9-dataset/Dataset/training dataset/Deals_to_Properties.csv")
train_acc_prop = pd.read_csv("../input/bb020fc4-9-dataset/Dataset/training dataset/Accounts_properties.csv")

In [ ]:
test_data = test.merge(train_acc , on='id_accs')
test_data = test_data.reset_index(drop = True)

In [ ]:
test_data.investor_type.value_counts()

In [ ]:
# test_data.head(29)

In [ ]:
train_data = train_acc.merge(train_opp , on='id_accs')
train_data.shape

In [ ]:
train_data = train_data.merge(train_deal[['id_deals','id_props']] , on='id_deals')
train_data.shape

In [ ]:
len(train_data.id_accs.unique())

In [ ]:
sample = train_data.loc[(train_data.investor_type == 'Advisor/Equity Fund')]

In [ ]:
sample.head()

In [ ]:
sample1= sample.loc[sample.foreign]

In [ ]:
sample.loc[sample.id_accs == '0012A000023XlQFQA0']

In [ ]:
sample.groupby(['id_accs'])[['id_props']].agg(['count'])

In [ ]:
train_data_1 = train_data[train_acc.columns ]

In [ ]:
train_data_1 = train_data_1.drop_duplicates()

In [ ]:
test_data_1 = test_data

In [ ]:
train_data_1 = train_data_1.reset_index(drop=True)

In [ ]:
trainip = train_data_1['investor_type']
testip = test_data_1['investor_type']
train_id= train_data_1['id_accs']
test_id = test_data_1['id_accs']

In [ ]:
train_data_1 = train_data_1.drop(['id_accs'], axis= 1)
test_data_1 = test_data_1.drop(['id_accs'], axis= 1)

In [ ]:
numeric_features = ['active_deals','activity_count','num_deals_as_client','num_deals_as_investor',
                   'number_of_properties','number_of_related_deals','number_of_related_properties',
                   'number_of_won_deals_as_client']

In [ ]:
train_numeric = train_data_1[numeric_features]
train_object = train_data_1.drop(numeric_features, axis=1)
test_numeric = test_data_1[numeric_features]
test_object = test_data_1.drop(numeric_features, axis=1)

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for col in train_object:
    if train_object[col].dtype == 'bool':
        # If 2 or fewer unique categories
        if len(list(train_object[col].unique())) <= 2:
            # Train on the training data
            le.fit(train_object[col])
            # Transform both training and testing data
            train_object[col] = le.transform(train_object[col])
            test_object[col] = le.transform(test_object[col])

In [ ]:
train_object = pd.get_dummies(train_object)
test_object = pd.get_dummies(test_object)

In [ ]:
# train_data_1 =train_data_1.dropna(subset=['investor_type'])

In [ ]:
# lbl = LabelEncoder()
# lbl.fit(list(train_data_1['investor_type'].unique())+list(test_data_1['investor_type'].unique()))
# train_data_1['investor_type'] = lbl.transform(train_data_1['investor_type'])
# test_data_1['investor_type'] = lbl.transform(test_data_1['investor_type'])


In [ ]:
print(train_object.shape)
print(test_object.shape)

In [ ]:
missing_cols = set( train_object.columns ) - set( test_object.columns )
for c in missing_cols:
    test_object[c] = 0
test_object = test_object[train_object.columns]
test_object.shape

In [ ]:
print(train_object.shape)
print(test_object.shape)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler1= StandardScaler().fit(train_numeric)
xtransform = scaler1.transform(train_numeric)
scaler2= StandardScaler().fit(test_numeric)
ytransform = scaler1.transform(test_numeric)

In [ ]:
train_numeric = pd.DataFrame(xtransform, columns=numeric_features) 
test_numeric = pd.DataFrame(ytransform, columns=numeric_features) 

In [ ]:
train = train_numeric.join(train_object)
test = test_numeric.join(test_object)

In [ ]:
print(train.shape)
print(test.shape)

In [ ]:
test['investor_type']=testip
train['investor_type']= trainip
train['id_accs'] = train_id

In [ ]:
def cosine_similarity(i , group , tes  ):
    print('=' * 80)
    u = tes.iloc[i]
    invest_type = u.investor_type
    print("ACCOUNT TYPE = ",invest_type)
    preference_list = []
    if u.buyer_book == 1 :
        preference_list.append("Investment Sales")
    if u.lender_book == 1:
        preference_list.append("Debt")
    if u.loan_sales_book == 1:
        preference_list.append("Loan Sales")
    
#      print('=' * 80)
    print("ACCOUNT PREFERENCE = " , preference_list)
    
    sample_test = tes.drop(['investor_type'], axis=1)
    w = sample_test.iloc[i]
    
    train_group = group.loc[group.investor_type== invest_type]
    if u.foreign == 0:
        train_group = train_group[train_group.foreign !=1]
    else:
        train_group = train_group[train_group.foreign ==1]
        
    if u.servicing_contract == 0:
        train_group = train_group[train_group.servicing_contract ==0]
#     else:
#         train_group = train_group[train_group.servicing_contract ==0]
#     print("shape of dataframe",train_group.shape )
    train_id = train_group.id_accs
    train_group = train_group.drop(['id_accs','investor_type'], axis=1)
#     print("shape of dataframe after drop",train_group.shape )
    train_group = train_group.reset_index(drop=True)
    cos_sim_list = []
    norm_u = np.linalg.norm(w)
    for i in range(0, train_group.shape[0]):
        v = train_group.iloc[i]
        dot = np.dot(w, v)
        norm_v = np.linalg.norm(v)
        cos_sim = (dot/norm_u * norm_v)
        cos_sim_list.append(cos_sim)
    output = pd.DataFrame({'train_id': train_id})
    output['cos_sim_list'] = cos_sim_list

    return preference_list , output 

In [ ]:
def  most_popular(group,  n_max=250):
    relevance = group['cos_sim_list'].values
    deals_cluster = group['train_id'].values
    popular = deals_cluster[np.argsort(relevance)[::-1]][:n_max]
    
    return popular

In [ ]:
properties.loc[(properties['property_type_1']=='Office') & (properties['num_buildings'].isnull()), 'num_buildings'] = 1
properties.loc[(properties['property_type_1']=='Office') & (properties['num_floors'].isnull()), 'num_floors'] = 6
properties.loc[(properties['property_type_1']=='Office') & (properties['num_parking_spaces'].isnull()), 'num_parking_spaces'] = 506

In [ ]:
properties.loc[(properties['property_type_1']=='Industrial') & (properties['num_buildings'].isnull()), 'num_buildings'] = 1
properties.loc[(properties['property_type_1']=='Industrial') & (properties['num_floors'].isnull()), 'num_floors'] = 1
properties.loc[(properties['property_type_1']=='Industrial') & (properties['num_parking_spaces'].isnull()), 'num_parking_spaces'] = 160

In [ ]:
properties.loc[(properties['property_type_1']=='Retail') & (properties['num_buildings'].isnull()), 'num_buildings'] = 2
properties.loc[(properties['property_type_1']=='Retail') & (properties['num_floors'].isnull()), 'num_floors'] = 2
properties.loc[(properties['property_type_1']=='Retail') & (properties['num_parking_spaces'].isnull()), 'num_parking_spaces'] = 513

In [ ]:
properties.loc[(properties['property_type_1']=='Industrial') & (properties['num_buildings'].isnull()), 'num_buildings'] = 1

In [ ]:
properties.loc[(properties['property_type_1']=='Multi-Housing') & (properties['num_buildings'].isnull()), 'num_buildings'] = 12
properties.loc[(properties['property_type_1']=='Multi-Housing') & (properties['num_floors'].isnull()), 'num_floors'] = 4
properties.loc[(properties['property_type_1']=='Multi-Housing') & (properties['num_parking_spaces'].isnull()), 'num_parking_spaces'] = 351

In [ ]:
properties.loc[(properties['property_type_1']=='Hotel-Lodging') & (properties['num_buildings'].isnull()), 'num_buildings'] = 6
properties.loc[(properties['property_type_1']=='Hotel-Lodging') & (properties['num_floors'].isnull()), 'num_floors'] = 6
properties.loc[(properties['property_type_1']=='Hotel-Lodging') & (properties['num_parking_spaces'].isnull()), 'num_parking_spaces'] = 245

In [ ]:
ft = properties.loc[properties.property_type_1 =='Retail']
ft.head(20)

In [ ]:
build = properties.loc[properties['num_parking_spaces'].isnull()]

In [ ]:
properties['num_floors']= properties['num_floors'].fillna(2)
properties['num_buildings']=properties['num_buildings'].fillna(2)
properties['num_parking_spaces']=properties['num_parking_spaces'].fillna(96)


In [ ]:
properties.loc[(properties['num_buildings']== 1) & (properties['num_floors'] ==1)& (properties['property_type_1'].isnull()), 'property_type_1'] = 'Industrial'

In [ ]:
properties.loc[(properties['num_buildings']== 1) & (properties['num_floors'] >=3)& (properties['property_type_1'].isnull()), 'property_type_1'] = 'Office'

In [ ]:
properties.loc[(properties['num_buildings'] >=1) & (properties['num_floors'] <=2)& (properties['property_type_1'].isnull()), 'property_type_1'] = 'Retail'

In [ ]:
properties.loc[(properties['num_buildings'] >=2) & (properties['num_floors'] <=5) & (properties['property_type_1'].isnull()), 'property_type_1'] = 'Multi-Housing'

In [ ]:
properties.loc[(properties['num_buildings'] >=2) & (properties['num_floors'] >=10)  & (properties['property_type_1'].isnull()), 'property_type_1'] = 'Multi-Housing'

In [ ]:
fc = properties.loc[properties.property_type_1.isnull()]
fc.shape

In [ ]:
properties['property_type_1']=properties['property_type_1'].fillna('Office')

In [ ]:

properties['class'] = properties['class'].fillna('no_class')
properties['region__c'] = properties['region__c'].fillna('Southwest')

In [ ]:
properties1 = properties.drop(['building_status','building_tax_expenses','id_deals','city','county','sale_status','market','occupancy_rate',
                               'price_per_sq_ft','price_per_unit','sale_amount__c','sale_date__c','size_acres','size_sf','size_units','year_built'], axis=1)

In [ ]:
p_id = properties1['property_type_1'] 

In [ ]:
le.fit(properties1['portfolio'])
properties1['portfolio'] = le.transform(properties1['portfolio'])
properties1['portfolio'].value_counts()

In [ ]:
properties1 =  pd.get_dummies(properties1,  columns=['class', 'property_type_1','region__c'])

In [ ]:
properties1[['num_buildings', 'num_floors','num_parking_spaces']] = StandardScaler().fit_transform(properties1[['num_buildings', 'num_floors','num_parking_spaces']])

In [ ]:
properties1.head()

In [ ]:
properties1['property_type_1'] = p_id 

In [ ]:
train_deal.shape

In [ ]:
properties_x = properties1[~(properties1.id_props.isin(train_deal.id_props))]
properties_x.shape

In [ ]:
# gfgf

In [ ]:
properties_y = properties1[(properties1.id_props.isin(train_deal.id_props))]
properties_y.shape

In [ ]:
properties_y.property_type_1.value_counts()

In [ ]:
properties_x.property_type_1.value_counts()

In [ ]:
# test_data.head(29)

In [ ]:
def  most_popular1(group,  n_max=250):
    relevance = group['cos_sim_list'].values
    deals_cluster = group['id_props'].values
    popular = deals_cluster[np.argsort(relevance)[::-1]][:n_max]
    
    return popular

In [ ]:
def cosine_similarity1(i , group , tes ):
    train_group = group.drop(['id_props','property_type_1'], axis=1)
    test_group = tes.drop(['id_props','property_type_1'], axis=1)
    u = test_group.iloc[i]
    cos_sim_list = []
    norm_u = np.linalg.norm(u)
    for i in range(0, train_group.shape[0]):
        v = train_group.iloc[i]
        dot = np.dot(u, v)
        norm_v = np.linalg.norm(v)
        cos_sim = (dot/norm_u * norm_v)
        cos_sim_list.append(cos_sim)

    return cos_sim_list

In [ ]:
def return_similar_prop(pref_list ,evalt ,properties_x ,properties_y ):
    my_dict2 = dict()
    for i in pref_list : 
        print("Preference Platform : ",i)
        my_dict1 = dict()
        sample = evalt.loc[evalt.platform == i]
        property_type_list = list(set(sample.property_group.unique()))
        
        for j  in  property_type_list:
            print("------------------------------ Property Type : ",j)
            sample1 = sample.loc[sample.property_group== j]
            prop = sample1.id_props
            similar_prop = get_similar_prop(j , prop, properties_x,properties_y)
            key1= j
            my_dict1[key1]=similar_prop
        flat_list = [item for sublist in my_dict1.values() for item in sublist]
        flat_list = list(set(flat_list))
#         print(len(flat_list))
#         print(flat_list[:3])
        
        key2 = i
        my_dict2[key2]=flat_list 
    flat_list1 = [item for sublist in my_dict2.values() for item in sublist]
    flat_list1 = list(set(flat_list1))
    print(len(flat_list1))
    return flat_list1

In [ ]:
def get_similar_prop( property_type , prop_list , properties_x ,properties_y):
    if property_type =='Mixed Use':
        properties_s = properties_x
    else: 
        properties_s = properties_x.loc[properties_x.property_type_1 == property_type ]
        properties_s =properties_s.reset_index(drop=True)
    my_dict1 = dict()
    for i in prop_list :
        j = properties_y.index[properties_y.id_props ==i]
        cos_sim_prop = cosine_similarity1(j , properties_s, properties_y)
        result  = pd.DataFrame({'id_props': properties_s.id_props})
        result['cos_sim_list'] = cos_sim_prop

        ft3 = 'id_props'
        most_pop_prop = most_popular1(result ,1)
        key1= i
        my_dict1[key1]=most_pop_prop
    flat_list = [item for sublist in my_dict1.values() for item in sublist]
    flat_list = list(set(flat_list))
    print(len(flat_list))
    print("---------------------------------Similar Prop: ",flat_list[:3])
    return flat_list 

In [ ]:
my_dict=dict()
import random


for i in range(test_data.shape[0]):
    print("ACC_ID:- ", i )
    pref_list ,cos_sim_prop = cosine_similarity(i ,train, test )

    cos_sim_prop= cos_sim_prop.sort_values(by=['cos_sim_list'],ascending=False) 

    pop_acc = most_popular(cos_sim_prop ,2)
    print('\n')
    print("SIMILAR ACCOUNTS = ",pop_acc)
    evalt = train_data[(train_data.id_accs.isin(pop_acc))]
    sample = evalt.loc[evalt.platform == i]
    property_type_list = list(set(sample.property_group.unique()))


    similar_prop = return_similar_prop(pref_list ,evalt, properties_x ,properties_y)
    print(similar_prop[:3])
#     random.shuffle(similar_prop)

#     n = [30,16,15,25,12,16,25,16,12,3,11,14,9,4,9,15,16,23,7,13,26,14,3,13,2,4,26,16,20]
    for prop in similar_prop[:3]:
        print("Deleting Allocated records ---->")
        properties_x.drop(properties_x[properties_x.id_props == prop].index, inplace=True)

    key= test_id[i]
    my_dict[key]=similar_prop[:3]

In [ ]:
properties_x.shape

In [ ]:
df1 = pd.DataFrame.from_dict(my_dict, orient = "index").stack().astype(object).reset_index(level=1, drop=True).reset_index()

In [ ]:
df1.rename(columns={'index':'id_accs',0:'id_prop'}, inplace=True)
df1.shape

In [ ]:
df1.head()

In [ ]:
df1.id_accs.value_counts()

In [ ]:
df1.to_csv('beta5.csv', index=False)